#### Importes

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import pickle
from sklearn.feature_selection import f_classif, SelectKBest
from scipy.stats import chi2_contingency
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import f_oneway
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



### Comprensión empresarial

Los depósitos a largo plazo permiten a los bancos retener dinero durante un período de tiempo específico, lo que permite al banco utilizar ese dinero para mejorar sus inversiones. Las campañas de marketing de este producto se basan en llamadas telefónicas. Si un usuario no se encuentra disponible en un momento dado, entonces se le volverá a llamar de nuevo en otro momento.

Descripción del problema

El banco portugués está teniendo una disminución en sus ingresos, por lo que quieren poder identificar a los clientes existentes que tienen una mayor probabilidad de contratar un depósito a largo plazo. Esto permitirá que el banco centre sus esfuerzos de marketing en esos clientes y evitará perder dinero y tiempo en clientes que probablemente no se suscribirán.

Para abordar este problema crearemos un algoritmo de clasificación que ayude a predecir si un cliente contratará o no un depósito a largo plazo.

- age. Edad del cliente (numérico)

- job. Tipo de trabajo (categórico)

- marital. Estado civil (categórico)

- education. Nivel de educación (categórico) 

- default. ¿Tiene crédito actualmente? (categórico) / FUERA

- housing. ¿Tiene un préstamo de vivienda? (categórico)

- loan. ¿Tiene un préstamo personal? (categórico)

- contact. Tipo de comunicación de contacto (categórico) /FUERA

- month. Último mes en el que se le ha contactado (categórico) 

- day_of_week. Último día en el que se le ha contactado (categórico)

- duration. Duración del contacto previo en segundos (numérico) / FUERA

- campaign. Número de contactos realizados durante esta campaña al cliente (numérico) / FUERA

- pdays. Número de días que transcurrieron desde la última campaña hasta que fue contactado (numérico) / FUERA

- previous. Número de contactos realizados durante la campaña anterior al cliente (numérico) /FUERA

- poutcome. Resultado de la campaña de marketing anterior (categórico) / FUERA

- emp.var.rate. Tasa de variación del empleo. Indicador trimestral (numérico)

- cons.price.idx. Índice de precios al consumidor. Indicador mensual (numérico)

- cons.conf.idx. Índice de confianza del consumidor. Indicador mensual (numérico) 

- euribor3m. Tasa EURIBOR 3 meses. Indicador diario (numérico) 

- nr.employed. Número de empleados. Indicador trimestral (numérico) /FUERA 'Necesito datos de los clientes, no de los chambeadores'

- y. TARGET. El cliente contrata un depósito a largo plazo o no (categórico) 


## PASO 1 : Planteamos nuestro problema o nuestro target a investigar


#### ¿Que cliente contratará o no un depósito a largo plazo?

## PASO 2: Recopilacion de datos

Recopilamos la informacion de nuestro DataSet

In [ ]:
df = pd.read_csv("/workspaces/machine-learning-elius123ef/data/raw/bank-marketing-campaign-data.csv", sep=";")

df.head()

## Paso 3: Análisis Descriptivo

In [ ]:
df.shape

In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df.columns

## Paso 4: Limpieza de Datos

### Normalizar valores de texto

In [ ]:

for col in ['job','marital','education','default','housing','loan','y']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower()
col

### Mapear target


In [ ]:
df['y_bin'] = df['y'].map({'yes': 1, 'no': 0})
df['y'].value_counts(dropna=False)

In [ ]:
df['y_bin'].value_counts(dropna=False)

### Buscamos valores duplicados:

In [ ]:
df.duplicated().sum()

##### Observaciones:

No encontramos valores duplicados.

### Buscamos valores nulos o sin conocer 

#### Nulos por columna


In [ ]:
df.isna().sum().sort_values(ascending=False)

### Limpieza de datos: Eliminar información irrelevante



#### Buscamos que informacion es irrelevante para nosotros y asi proceder a eliminarla

In [ ]:
df.columns

### Eliminacion de columnas innecesarias:

**NOTA: Nos aseguramos de hacer una copia para asi no perder el df original.**

In [ ]:
df_copy = df.copy()
df_copy

In [ ]:
df_copy.drop(["contact", "month", "day_of_week", "campaign", "pdays", "poutcome",
            "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"], axis=1, inplace=True)
df_copy

#### Observaciones

- Eliminamos las columnas `contact` `month` `day_of_week` `campaign` `pdays` `poutcome` `emp.var.rate` `cons.price.idx` `cons.conf.idx` `euribor3m` `nr.employed` , ya que considero que son variables que no influirian en la decision de algun cliente, y que influye mas en las relaciones entre los bancos.

## Paso 5: Análisis de Variables

### Análisis de Variables Univariante

### Análisis de Variables Univariante Categóricas

In [ ]:
df_copy.info()

#### Hacemos nuestro análisis univariante numericas:

Pasos que seguiremos:

- Histogramas, boxplots.

- Medidas de tendencia central (media, mediana) y dispersión (varianza, desviación).

- Outliers (IQR, z-score).


##### Buscamos outliers

### Hacemos nuestro análisis univariante Categóricas:

con los siguientes pasos a seguir :

1. Frecuencias absolutas y relativas
2. Tablas de frecuencia 
3. Visualización con gráfico
4. Medidas de concentración y diversidad

#### Nuestra variables categorica son:

1. job
2. marital
3. education
4. default
5. housing
6. loan
7. y

Para simplificar la tarea, creamos este pequeño bucle que nos aportara la informacion que queremos ver, de cada variable

In [ ]:
categorical_vars = ['job','marital','education','default','housing','loan','y']

for col in categorical_vars:
    print(f"\nVariable: {col}")
    print(df[col].value_counts(normalize=True) * 100)

    # Gráfico
    plt.figure(figsize=(8,4))
    sns.countplot(x=col, data=df,hue='y', order=df_copy[col].value_counts().index , palette='dark:orange')
    plt.xticks(rotation=45)
    plt.grid()
    plt.title(f"Distribución de {col}")
    plt.show()

    # Tabla cruzada con y_bin
    pd.crosstab(df_copy[col], df_copy['y_bin'], normalize='index')

    # Chi-cuadrado
    table = pd.crosstab(df[col], df_copy['y_bin'])
    chi2, p, dof, expected = chi2_contingency(table)
    print(f"Chi-cuadrado p-valor: {p}")


##### Observado los anteriores graficos, hemos decidido que:

- Variables muy útiles (significativas): job, education, marital, default.

- Variables poco útiles (no significativas): housing, loan.

**Precaución:**

- default tiene categorías muy desbalanceadas puede necesitar recodificación.

- y está desbalanceada aplicar técnicas de balanceo antes de entrenar.


Por lo cual descartaremos:

-  housing p-valor ≈ 0.058, no significativa.

-  loan p-valor ≈ 0.58, sin relación con y_bin.


### Hacemos nuestro analisis análisis bivariante


In [ ]:
df_copy.info()

### Numérica vs. numérica:

In [ ]:
df_copy[['age','duration','previous']].corr()


#### - Heatmap de correlaciones:


In [ ]:
sns.heatmap(df_copy[['age','duration','previous']].corr(), annot=True, cmap='coolwarm')
plt.show()


##### Conclusion

- No hay relaciones lineales fuertes entre las variables numéricas.
- Puedes usarlas todas en el modelo sin preocuparte por redundancia.
- La clave será analizar cómo cada una se asocia con y_bin (ej. duración de la llamada suele ser más relevante para predecir aceptación).


####  Gráficos de dispersión


In [ ]:
sns.pairplot(df_copy[['age','duration','previous','y_bin']], hue='y_bin')
plt.show()

##### Conclusion

- Duration es clave: las llamadas largas están más asociadas con respuestas positivas.
- Age y Previous aportan menos por sí solas, pero pueden ser relevantes en combinación con variables categóricas.
- El análisis bivariante numérico vs numérico ayuda a decidir qué variables numéricas son más prometedoras para el modelo.


### Numérica vs. categórica:



- Boxplots / Violin plots:


In [ ]:
fig, axis = plt.subplots(figsize=(10, 5), ncols=2)
sns.boxplot(ax=axis[0], x='job', y='age', data=df_copy)
sns.violinplot(ax=axis[1], x='marital', y='duration', data=df_copy)
plt.tight_layout()
plt.show()

##### Conclusion

- El boxplot de age vs job confirma una relación lógica y significativa: la edad está condicionada por el tipo de trabajo.
- El violin plot de duration vs marital muestra que la duración de la llamada tiene una distribución similar entre estados civiles, aunque con algunos outliers que podrían ser relevantes para el target.


- ANOVA / t-test:
Evalúa si las medias de la variable numérica difieren significativamente entre categorías.


In [ ]:
t_test = f_oneway(*[df_copy.loc[df_copy['marital']==cat,'age'] for cat in df_copy['marital'].unique()])
t_test



### Categórica vs. categórica:



- Tablas de contingencia:


In [ ]:
pd.crosstab(df_copy['job'], df_copy['y_bin'], normalize='index')


#### Gráficos de barras apiladas:





In [ ]:
sns.countplot(x='education', hue='y_bin', data=df_copy)
plt.xticks(rotation=45)
plt.show()

##### Conclusion

- La educación está asociada con la respuesta, pero la tendencia general es que la mayoría rechaza la campaña.

- Chi-cuadrado:


In [ ]:
table = pd.crosstab(df_copy['marital'], df_copy['y_bin'])
chi2, p, dof, expected = chi2_contingency(table)
f"p-valor: {p}"


##### Conclusion

Un p‑valor tan bajo como 2e‑26 indica que la asociación entre esas variables es real y muy fuerte, no atribuible al azar.
Esto convierte a esa variable en una candidata importante para tu modelo predictivo



### Análisis multivariante

#### Matriz de correlación ampliada


In [ ]:
vars_num = ['age', 'duration', 'previous', 'y_bin']
corr_matrix = df_copy[vars_num].corr()

plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title("Matriz de correlación ampliada con y_bin")
plt.show()


##### Conclusion

- Las numéricas son independientes entre sí.
- Solo duration muestra una relación clara con el target.
- Esto confirma que en el modelado debería dar más peso a duration, mientras que age y previous pueden ser secundarios o útiles en interacciones.


#### Multicolinealidad (VIF)


In [ ]:
X = df_copy[['age', 'duration', 'previous']]
X_const = add_constant(X)

vif_data = pd.DataFrame()
vif_data['Variable'] = X_const.columns
vif_data['VIF'] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]

vif_data


##### Conclusion

- Todos los VIF están muy cerca de 1, lo que indica que ninguna variable numérica está correlacionada con las otras.

- El paso de VIF confirma que se puede usar age, duration y previous juntas en el modelo sin riesgo de redundancia.

- La informacion que aportan es independiente y complementaria.


### Interacciones entre variables

#### Gráficos de barras agrupada

In [ ]:
sns.catplot(x='education', hue='marital', col='y_bin', data=df_copy, kind='count', height=4, aspect=1.5)
plt.suptitle("Interacción: Educación vs Estado Civil según y_bin", y=1.05)
plt.show()


##### Conclusion

Este grafico nos permite:

- Detectar segmentos de clientes más propensos a aceptar.
- Diseñar campañas más efectivas dirigidas a esos perfiles


#### Boxplots segmentados

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='job', y='duration', hue='loan', data=df_copy)
plt.title("Interacción: Duración vs Job según préstamo")
plt.xticks(rotation=45)
plt.show()


##### Conclusion

- Esta interacción sugiere que el efecto de la duración sobre la aceptación puede depender del tipo de trabajo y del estado de préstamo.

- Este gráfico confirma que duration no actúa igual en todos los perfiles.
- El tipo de trabajo y el estado de préstamo modulan su impacto, lo que puede ser clave para segmentar clientes o mejorar el modelo predictivo.


#### Tablas cruzadas con múltiples variables

In [ ]:
cross_tab = pd.crosstab([df_copy['job'], df_copy['education']], df_copy['y_bin'], normalize='index')
cross_tab.round(3)



##### Conclusion

- La interacción entre job y education sí influye significativamente en el target y_bin.
- Hay combinaciones que destacan por su mayor tasa de aceptación, lo que puede ser clave para segmentar clientes o enriquecer el modelo predictivo.
- Este tipo de análisis te permite pasar de correlaciones simples a patrones de comportamiento más complejos y útiles.


#### Boxplot: duración vs estado civil segmentado por housing

In [ ]:

plt.figure(figsize=(10,6))
sns.boxplot(x='marital', y='duration', hue='housing', data=df_copy)
plt.title("Interacción: Duración vs Marital según housing")
plt.show()


##### Conclusion

- El gráfico confirma que duration no actúa igual en todos los perfiles.
- El estado civil y el estado de vivienda modulan su impacto, lo que puede ser clave para segmentar clientes o mejorar el modelo predictivo.


 ### Visualización multivariante


####  Pairplot  de variables numericas


In [ ]:
sns.pairplot(df_copy, vars=['age', 'duration', 'previous'], hue='y_bin', palette='coolwarm')
plt.show()

##### Conclusion

- Muestra cómo se distribuyen las variables numéricas entre sí.
- Coloreado por y_bin para ver si hay agrupaciones o separaciones.
- Esperamos que duration muestre mayor diferenciación entre clases.


#### Heatmap de correlaciones




##### factorizamos 

In [ ]:
df_copy['job'] = pd.factorize(df['job'])[0]
df_copy['marital'] = pd.factorize(df['marital'])[0]
df_copy['default'] = pd.factorize(df['default'])[0]
df_copy['housing'] = pd.factorize(df['housing'])[0]
df_copy['loan'] = pd.factorize(df['loan'])[0]

##### Visualizamos el Heatmap con todas las variables

In [ ]:
corr = df_copy[["age", "job", "marital", "default", "housing", "loan", "duration", "previous", "y_bin"]].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

fig, axis = plt.subplots(figsize=(10, 6))
sns.heatmap(corr, mask=mask, annot=True, linewidths=0.5, fmt=".2f", center=0)

plt.tight_layout()
plt.show()

##### Conclusion

- Las variables más predictivas para y_bin son duration y previous.
- Las variables categóricas aportan poco por sí solas, pero podrían ser útiles en interacciones.
- Hay poca multicolinealidad, lo que es bueno para la estabilidad del modelo.


## Paso 6: Ingeniería de características

#### Crear nuevas variables (ej. binning de edad, ratios).

In [ ]:
df_copy['duration_bin'] = pd.cut(df_copy['duration'],
                            bins=[0, 200, 600, 1500, df['duration'].max()],
                            labels=['muy_corta', 'corta', 'media', 'larga'])


- Decisiones tomadas tras el EDA:
- Agrupar categorías raras.
- Crear nuevas variables (ej. binning de edad, ratios).
- Transformaciones (log, normalización).
- Codificación inicial (One-Hot, Target Encoding)


#### Codificación de categóricas


In [ ]:
num_vars = ['age', 'duration', 'previous']
cat_vars = ['job', 'education', 'marital', 'housing', 'loan', 'default', 'duration_bin']
df_encoded = pd.get_dummies(df_copy[cat_vars], drop_first=True)
df_encoded

#### Interacción: job + education


In [ ]:
df_copy['job_edu'] = df_copy['job'].astype(str) + "_" + df_copy['education'].astype(str)
df_job_edu = pd.get_dummies(df_copy['job_edu'], drop_first=True)


#### Variables derivadas


In [ ]:
df_copy['contact_intensity'] = df_copy['previous'] / df_copy['duration'].replace(0, 1)  # evitar división por cero
df_copy['is_retired_no_loan'] = ((df_copy['job'] == 'retired') & (df_copy['loan'] == 'no')).astype(int)
df_copy['is_single_high_edu'] = ((df_copy['marital'] == 'single') & (df_copy['education'] == 'university.degree')).astype(int)



#### Escalado de numéricas


In [ ]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_copy[num_vars + ['contact_intensity']]),
                         columns=num_vars + ['contact_intensity'])


#### Unimos todo

In [ ]:
X_final = pd.concat([df_scaled, df_encoded, df_job_edu, df_copy[['is_retired_no_loan', 'is_single_high_edu']]], axis=1)
y_final = df_copy['y']

### Detección de patrones y outliers

- Outliers en numéricas (boxplots, z-score).

- Categorías raras en categóricas (frecuencia <1%).

- Distribuciones sesgadas (skewness, kurtosis).


## Paso 7: Split (dos métodos o enfoques)

#### División en train/test (ej. 80/20).


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y_final, test_size=0.3, random_state=42, stratify=y_final)


#### Entrenar y modular

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


####  Predicciones



In [ ]:
y_pred = model.predict(X_test)


#### Métricas de evaluación


In [351]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='yes')
recall = recall_score(y_test, y_pred, pos_label='yes')
f1 = f1_score(y_test, y_pred, pos_label='yes')

accuracy, precision, recall, f1


(0.8978716516953953,
 0.6105442176870748,
 0.2579022988505747,
 0.36262626262626263)

##### Conclusion

1. Accuracy: El modelo acierta en casi el 90% de los casos. Pero cuidado: si las clases están desbalanceadas, esta métrica puede ser engañosa.

2. Precision: De todas las veces que el modelo predijo “sí” (y = yes), solo el 60.6% eran correctas. Hay bastantes falsos positivos.

3. Recall: Solo detecta el 27.4% de los verdaderos “sí”. Es decir, se le escapan muchos casos positivos.

4. F1 Score: Promedio armónico entre precisión y recall. Indica que el modelo tiene dificultades para capturar correctamente los positivos


## Paso 8: Selección de características

#### Univariate Selection (SelectKBest + chi2 o f_classif)

In [353]:
constant_cols = [col for col in X_final.columns if X_final[col].nunique() == 1]
constant_cols

['is_retired_no_loan', 'is_single_high_edu']

In [354]:
X_clean = X_final.drop(columns=constant_cols)


In [355]:
selector = SelectKBest(score_func=f_classif, k=20)
X_selected = selector.fit_transform(X_clean, y_final)

selected_features = X_clean.columns[selector.get_support()]
selected_features


Index(['age', 'duration', 'previous', 'job', 'default', 'education_basic.9y',
       'education_university.degree', 'duration_bin_corta',
       'duration_bin_media', 'duration_bin_larga', '11_basic.9y',
       '11_high.school', '11_unknown', '1_high.school', '2_university.degree',
       '3_basic.4y', '3_basic.9y', '5_basic.4y', '5_university.degree',
       '5_unknown'],
      dtype='object')

In [356]:
X_selected = X_clean[selected_features]


In [357]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_final, test_size=0.3, random_state=42, stratify=y_final)


In [362]:
model = LogisticRegression(max_iter=1000,  class_weight='balanced')
model.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [363]:
y_pred = model.predict(X_test)

In [364]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='yes')
recall = recall_score(y_test, y_pred, pos_label='yes')
f1 = f1_score(y_test, y_pred, pos_label='yes')

accuracy, precision, recall, f1

(0.8092579104960751,
 0.34430461439173926,
 0.7665229885057471,
 0.4751725673569361)

#### Comparación de resultados


       Métrica                Sin balanceo               Con class_weight='balanced'                                Diferencia clave
      Accuracy                  0.8979                           0.8093                   Bajó porque el modelo ahora se equivoca más en la clase mayoritaria.    
      Precision                 0.6105                           0.3443                   Más falsos positivos: el modelo predice más “yes” que no lo son.
      Recall                    0.2579                           0.7665                   ¡Gran mejora! El modelo detecta muchos más verdaderos “yes”.
      F1 Score                  0.3626                           0.4752                             Mejor balance entre precisión y recall.

#### Conclusión:


- Sin balanceo: el modelo era conservador, predecía pocos “yes” pero con más precisión. Sin embargo, se le escapaban muchos verdaderos positivos (recall bajo).
- Con balanceo: el modelo se volvió más sensible a la clase minoritaria (y = yes), lo que mejoró drásticamente el recall, aunque sacrificó precisión y accuracy


#### ¿Cuál elegir?
Depende del objetivo:
- Si tu prioridad es detectar la mayor cantidad de “yes” posibles (por ejemplo, clientes que aceptan una oferta), el modelo con class_weight='balanced' es mejor.
- Si prefieres evitar falsos positivos (por ejemplo, en fraude o riesgo), el modelo sin balanceo puede ser más útil.
